In [ ]:
import csv

def read_csv(file_path):
    data: list[dict] = []
    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        headers = next(reader)
        for row in reader:
            entry = {header: value for header, value in zip(headers, row)}
            data.append(entry)
    return data

# CSV extracted from: https://docs.google.com/spreadsheets/d/1L0_c53wjkWngAcv6e3GF2m6_zcVE2u8fd3XNNikcdTI
# with this header:
# name,number,b1,b2,b3,b4,b5,b6,b7,scrambled,scrambled_name,has_1_beat,has_time_travel,notes
read_csv(r"...")

In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\lukas\AppData\Local\Temp\Ambled V Variations - Ambled.csv")
df

In [ ]:
import re

mapping = {'A': 'C', 'B': 'A', 'C': 'B'}

def relabel(ch: str, times: int = 1) -> str:
  ch = ch.upper()
  for _ in range(times % 3):
    ch = mapping[ch]
  return ch

pattern = re.compile(r'^\s*([A-Za-z])\s*([ABC])\s*-\s*([ABC])\s*$')

def process_row(row):
  out = ['-'] * 7
  for i in range(1, 8):
    val = row.get(f'b{i}')
    if pd.isna(val):
      continue
    s = str(val).strip()
    m = pattern.match(s)
    if not m:
      continue
    kind, src, dst = m.group(1), m.group(2).upper(), m.group(3).upper()
    if kind == 'c':
      # skip carries
      continue
    shift = 1 if src == dst else 2
    wraps = 1 if i - shift <= 0 else 0
    new_idx = ((i - 1 - shift) % 7) + 1
    new_src = relabel(src, wraps)
    kind_suffix = '2' if kind == 'i' else ''
    token = f"{kind}{kind_suffix}{new_src}"
    pos = new_idx - 1
    if out[pos] == '-':
      out[pos] = token
    else:
      out[pos] = out[pos] + ' ' + token
  return out

df['b_processed'] = df.apply(process_row, axis=1)
df[['name', 'number', *(f'b{i}' for i in range(1, 8)), 'b_processed']]

In [ ]:
import json

# Convert b_processed lists to the required dict format
presets = []

for idx, row in df.iterrows():
  b_processed = row['b_processed']
  name = row['name'] if pd.notna(row['name']) else 'Unnamed'
  number = row['number']
  
  # Create instructions string from b_processed (join with spaces)
  instructions = ' '.join(b_processed)
  
  # Create the preset dict
  preset = {
    'name': f"{number} - {name}",
    'slug': re.sub(r'\W+', '-', str(name).lower()).strip('-'),
    'instructions': "RAW_DATA_WALKING_FEED_10C",
    'manipulators': [instructions]
  }
  
  presets.append(preset)

# Dump to JSON string
json_output = json.dumps(presets, indent=2)
print(json_output)